In [2]:
import os
import cv2
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam
import matplotlib as plt

def your_denoising_function(images):
    # Define the CNN model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=images[0].shape))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    # Train the model
    model.fit(images, images, epochs=10, batch_size=16)

    # Use the trained model to denoise the input images
    denoised_images = model.predict(images)

    return denoised_images


# Load the images from the specified folder
image_folder = './Test cases/'
images = []
for filename in os.listdir(image_folder):
    img = cv2.imread(os.path.join(image_folder,filename), 0)
    if img is not None:
        images.append(img)

# Convert the list of images to a numpy array
images = np.array(images)

# Normalize the images
images = images / 255.0

# Resize the images to the desired shape
height, width, channels = 256, 256, 1
resized_images = np.zeros((images.shape[0], height, width, channels))
for i in range(images.shape[0]):
    resized_images[i] = cv2.resize(images[i], (width, height)).reshape(height, width, channels)

# Apply the CNN denoising algorithm
denoised_images = your_denoising_function(resized_images)

# Visualize the results
for i in range(images.shape[0]):
    fig, ax = plt.subplots(1, 2)
    ax[0].imshow(images[i], cmap='gray')
    ax[0].set_title('Original')
    ax[1].imshow(denoised_images[i].squeeze(), cmap='gray')
    ax[1].set_title('Denoised')
    plt.show()


C:\Users\Jack\AppData\Local\Temp\ipykernel_1744\2763784586.py:44: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  images = np.array(images)


Epoch 1/10
13/13 [==============================] - 104s 7s/step - loss: 0.0719
Epoch 2/10
13/13 [==============================] - 106s 8s/step - loss: 0.0208
Epoch 3/10
13/13 [==============================] - 92s 7s/step - loss: 0.0093
Epoch 4/10
 9/13 [===================>..........] - ETA: 39s - loss: 0.0075

In [ ]:
import math
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

def compute_metrics(original_images, denoised_images):
    psnr_values = []
    ssim_values = []
    mse_values = []

    for i in range(original_images.shape[0]):
        # Compute PSNR
        mse = np.mean((original_images[i] - denoised_images[i])**2)
        if mse == 0:
            psnr = 100
        else:
            psnr = 20 * math.log10(255.0 / math.sqrt(mse))
        psnr_values.append(psnr)

        # Compute SSIM
        ssim_value = ssim(original_images[i], denoised_images[i], data_range=original_images[i].max() - original_images[i].min())
        ssim_values.append(ssim_value)

        # Compute MSE
        mse_value = mse = np.mean((original_images[i] - denoised_images[i])**2)
        mse_values.append(mse_value)

    return psnr_values, ssim_values, mse_values

# Load the original images from the specified folder
image_folder = './Test cases/'
original_images = []
for filename in os.listdir(image_folder):
    img = cv2.imread(os.path.join(image_folder,filename), 0)
    if img is not None:
        original_images.append(img)

# Convert the list of images to a numpy array
original_images = np.array(original_images)

# Normalize the original images
original_images = original_images / 255.0

# Resize the original images to the desired shape
height, width, channels = 256, 256, 1
resized_original_images = np.zeros((original_images.shape[0], height, width, channels))
for i in range(original_images.shape[0]):
    resized_original_images[i] = cv2.resize(original_images[i], (width, height)).reshape(height, width, channels)

# Apply the CNN denoising algorithm
denoised_images = your_denoising_function(resized_original_images)

# Compute the quantitative metrics
psnr_values, ssim_values, mse_values = compute_metrics(original_images, denoised_images)

# Print the results
for i in range(original_images.shape[0]):
    print('Image', i+1, 'PSNR:', round(psnr_values[i], 2), 'SSIM:', round(ssim_values[i], 2), 'MSE:', round(mse_values[i], 2))
